In [43]:
import pandas as pd
import sentier_data_tools as sdt
from datetime import datetime

In [44]:
sdt.reset_local_database()

In [45]:
metadata = sdt.Datapackage(
    name="electricity_markets",
    description="Electricity markets blabla",
    contributors=[
        {
            "title": "Peakachu",
            "path": "https://github.com/TimoDiepers/sentier_peakachu/",
            "role": "author",
        },
    ],
    homepage="https://github.com/TimoDiepers/sentier_peakachu/",
).metadata()

In [46]:
from sentier_peakachu.entsoe import get_generation_data
country_code = "PL"
df = get_generation_data(
    country_code=country_code,
    start=pd.Timestamp("20241008", tz="Europe/Brussels"),
    end=pd.Timestamp("20241009", tz="Europe/Brussels"),
)

In [47]:
df.index.name = "timestamp"
df = df[["Fossil Brown coal/Lignite", "Fossil Gas", "Solar"]].reset_index()
df.columns = ["timestamp", "https://example.com/coal", "https://example.com/gas", "https://example.com/pv"]

In [48]:
UNITS = ["https://example.com/units/datetime", "https://example.com/units/MW", "https://example.com/units/MW", "https://example.com/units/MW"]

In [49]:
dataset_id = sdt.Dataframe(
    name=f"electricity mixes",
    data=df,
    kind=sdt.DataframeKind.BOM,
    product="http://openenergy-platform.org/ontology/oeo/OEO_00000139",
    columns=[{"iri": x, "unit": y} for x, y in zip(df.columns, UNITS)],
    metadata=metadata,
    location=f"https://example.com/locations/{country_code}",
    version=1,
    valid_from=datetime(2018, 1, 1),
    valid_to=datetime(2028, 1, 1),
).save()

In [50]:
dataset = sdt.Dataframe.get_by_id(dataset_id)

In [51]:
list(sdt.Dataframe.select().where(sdt.Dataframe.product == "http://openenergy-platform.org/ontology/oeo/OEO_00000139"))

[<Dataframe: 1>]